In [1]:
import os
import glob

# Path to your dataset folder
# Each class should be in a separate folder inside this directory
dataset_path = "dataset"

# List all subfolders (classes)
classes = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

print("Found classes:", classes)

# Count images in each class
for cls in classes:
    class_path = os.path.join(dataset_path, cls)
    # Assuming common image extensions
    image_files = glob.glob(os.path.join(class_path, "*.[pjPJ]*[ngNG]*"))  # jpg, jpeg, png
    print(f"Class '{cls}' has {len(image_files)} images")


Found classes: ['Acne And Rosacea Photos', 'Malignant', 'Fu Athlete Foot', 'Eczema Photos', 'Fu Ringworm', 'Benign', 'Exanthems And Drug Eruptions', 'Lupus And Other Connective Tissue Diseases', 'Ba Impetigo', 'Bullous Disease Photos', 'Heathy', 'Light Diseases And Disorders Of Pigmentation', 'Atopic Dermatitis Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Melanoma Skin Cancer Nevi And Moles', 'Cellulitis Impetigo And Other Bacterial Infections', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Herpes Hpv And Other Stds Photos', 'Fu Nail Fungus', 'Ba  Cellulitis']
Class 'Acne And Rosacea Photos' has 6837 images
Class 'Malignant' has 6762 images
Class 'Fu Athlete Foot' has 8054 images
Class 'Eczema Photos' has 6715 images
Class 'Fu Ringworm' has 8129 images
Class 'Benign' has 6459 images
Class 'Exanthems And Drug Eruptions' has 7750 images
Class 'Lupus And Other Connective Tissue Diseases' has 7734 images
Class 'Ba Impetigo' has 8148 images
Class '

In [2]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [3]:
import os

# Suppress TensorFlow Python logs
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Suppress ML Compute / Metal debug info
os.environ["TF_METAL_ENABLE_LOGGING"] = "0"
os.environ["TF_GPU_THREAD_MODE"] = "gpu_private"

import tensorflow as tf
tf.debugging.set_log_device_placement(False)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import mixed_precision
import numpy as np
import os
import time

# --------------------------
# 1. Mixed Precision for M-series GPU
# --------------------------
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print("Compute dtype:", policy.compute_dtype)
print("Variable dtype:", policy.variable_dtype)

# --------------------------
# 2. Settings
# --------------------------
DATA_DIR = "dataset"  # All class folders inside
IMG_SIZE = (160, 160)  # Reduced resolution
BATCH_SIZE = 32         # Safe batch size
EPOCHS_FROZEN = 5
EPOCHS_FINE = 10
SEED = 42

# --------------------------
# 3. Detect classes safely
# --------------------------
classes = sorted([d for d in os.listdir(DATA_DIR) 
                  if os.path.isdir(os.path.join(DATA_DIR, d)) and not d.startswith('.')])
NUM_CLASSES = len(classes)
print("Detected classes:", classes)

# --------------------------
# 4. Data generators with augmentation
# --------------------------
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=10,
    brightness_range=[0.8, 1.2]
)

train_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED,
    classes=classes
)

val_gen = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED,
    classes=classes
)

# Convert to tf.data.Dataset for prefetching only
AUTOTUNE = tf.data.AUTOTUNE

train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen,
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_SIZE[0], IMG_SIZE[1], 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, NUM_CLASSES), dtype=tf.float32)
    )
).prefetch(AUTOTUNE)

val_ds = tf.data.Dataset.from_generator(
    lambda: val_gen,
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_SIZE[0], IMG_SIZE[1], 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, NUM_CLASSES), dtype=tf.float32)
    )
).prefetch(AUTOTUNE)

# --------------------------
# 5. Build model
# --------------------------
base_model = MobileNetV3Large(weights='weights_mobilenet_v3_large_224_1.0_float_no_top_v2.h5', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base_model.trainable = False  # Freeze backbone initially

x = GlobalAveragePooling2D()(base_model.output)
output = Dense(NUM_CLASSES, activation='softmax', dtype='float32')(x)  # float32 for stability

model = Model(inputs=base_model.input, outputs=output)
model.compile(optimizer=Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# --------------------------
# 6. Callbacks
# --------------------------
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
]

# --------------------------
# 7. Train frozen backbone
# --------------------------
start_time = time.time()
print("\n=== Training frozen backbone ===")
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FROZEN,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# 8. Fine-tune top layers safely
# --------------------------
print("\n=== Fine-tuning top layers ===")
base_model.trainable = True
# Freeze all but last 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(1e-5),  # lower LR for fine-tuning
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS_FINE,
    callbacks=callbacks,
    verbose=1
)

total_time = time.time() - start_time
print(f"\nTotal training time: {total_time/60:.2f} minutes")

# --------------------------
# 9. Prediction function
# --------------------------
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array/255.0, axis=0)
    
    pred_probs = model.predict(img_array, verbose=0)[0]
    best_idx = np.argmax(pred_probs)
    return classes[best_idx], dict(zip(classes, pred_probs))

# --------------------------
# 10. Example usage
# --------------------------
example_image = "dataset/Acne And Rosacea Photos/sample.jpg"
predicted_class, all_scores = predict_image(example_image)

print(f"\nPredicted class: {predicted_class}")
print("All class probabilities:", all_scores)


Compute dtype: float16
Variable dtype: float32
Detected classes: ['Acne And Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Atopic Dermatitis Photos', 'Ba  Cellulitis', 'Ba Impetigo', 'Benign', 'Bullous Disease Photos', 'Cellulitis Impetigo And Other Bacterial Infections', 'Eczema Photos', 'Exanthems And Drug Eruptions', 'Fu Athlete Foot', 'Fu Nail Fungus', 'Fu Ringworm', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Heathy', 'Herpes Hpv And Other Stds Photos', 'Light Diseases And Disorders Of Pigmentation', 'Lupus And Other Connective Tissue Diseases', 'Malignant', 'Melanoma Skin Cancer Nevi And Moles']
Found 121199 images belonging to 20 classes.
Found 30289 images belonging to 20 classes.

=== Training frozen backbone ===
Epoch 1/5
    403/Unknown 74s 174ms/step - accuracy: 0.0521 - loss: 3.0757